In [3]:
!pip install spacy beautifulsoup4
!pip install nlp


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!unzip ./OPP-115_v1_0.zip -d ./OPP-115


In [ ]:
import spacy
import os
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd

# Load Spacy model
nlp = spacy.load("en_core_web_sm")

# Preprocess HTML content
def preprocess_text(text):
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text(separator=" ")
    doc = nlp(clean_text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct]
    clean_text = " ".join(tokens)
    return clean_text

# Create a dictionary for preprocessed policies
preprocessed_policies_dict = {}
directory = './OPP-115/OPP-115/sanitized_policies'

for filename in os.listdir(directory):
    if filename.endswith('.html'):
        base_name = '_'.join(filename.split('_')[1:]).rsplit('.', 1)[0]
        path = os.path.join(directory, filename)
        with open(path, 'r', encoding='utf-8') as file:
            html_content = file.read()
            preprocessed_text = preprocess_text(html_content)
            preprocessed_policies_dict[base_name] = preprocessed_text

# Create dataframes from annotations
annotations_dir = './OPP-115/OPP-115/annotations/'
dataframes = {}

for annotation_filename in os.listdir(annotations_dir):
    if annotation_filename.endswith('.csv'):
        base_name = '_'.join(annotation_filename.split('_')[1:]).rsplit('.', 1)[0]
        annotation_path = os.path.join(annotations_dir, annotation_filename)
        annotation_df = pd.read_csv(annotation_path, header=None)
        dataframes[base_name] = {'categories': annotation_df[5], 'preprocessed_text': preprocessed_policies_dict.get(base_name)}

# Initialize a list to hold the final consolidated dataset
final_dataset = []

for base_name, data in dataframes.items():
    unique_categories_per_text = defaultdict(set)
    for category in data['categories']:
        unique_categories_per_text[data['preprocessed_text']].add(category)
    for text, categories_set in unique_categories_per_text.items():
        final_dataset.append({
            'source': base_name,
            'policies_text': text,
            'categories': list(categories_set)
        })

# Create the final DataFrame
final_df = pd.DataFrame(final_dataset)

# Map categories to GDPR principles
label_mapping = {
    "Other": [],
    "First Party Collection/Use": [1, 2, 3],
    "Third Party Sharing/Collection": [1, 2, 3],
    "Data Security": [6],
    "Policy Change": [1],
    "User Choice/Control": [1],
    "Data Retention": [5],
    "International and Specific Audiences": [1],
    "User Access, Edit and Deletion": [1, 4],
    "Do Not Track": []
}

def transform_document_labels(document_labels):
    binary_labels = [0] * 7
    for label in document_labels:
        if label in label_mapping:
            mapped_indices = label_mapping[label]
            for idx in mapped_indices:
                binary_labels[idx - 1] = 1
    return binary_labels

# Apply the label transformation
final_df['policies_labels'] = final_df['categories'].apply(transform_document_labels)

# Calculate compliance percentage
total_principles = 7
final_df['compliance_percentage'] = final_df['policies_labels'].apply(lambda x: (sum(x) / total_principles) * 100)

# Save the DataFrame
final_df.to_csv("./policies_gdpr_labels.csv", index=False)
final_df.head()

# Verify counts of GDPR principles
index_counts = [0] * 7
for labels in final_df['policies_labels']:
    for i in range(7):
        if labels[i] == 1:
            index_counts[i] += 1

# Display the results
for i, count in enumerate(index_counts):
    print(f"(GDPR Principle {i+1}) count: {count}")


In [ ]:
import spacy
import os
from bs4 import BeautifulSoup

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text(separator=" ")

    doc = nlp(clean_text)

    tokens = [token.lemma_.lower() for token in doc if not token.is_punct]

    clean_text = " ".join(tokens)
    return clean_text


preprocessed_policies_dict = {}
directory = './OPP-115/OPP-115/sanitized_policies'

for filename in os.listdir(directory):
    if filename.endswith('.html'):
        base_name = '_'.join(filename.split('_')[1:]).rsplit('.', 1)[0]
        path = os.path.join(directory, filename)
        with open(path, 'r', encoding='utf-8') as file:
            html_content = file.read()
            preprocessed_text = preprocess_text(html_content)
            preprocessed_policies_dict[base_name] = preprocessed_text


In [ ]:
from collections import defaultdict
import pandas as pd

annotations_dir = './OPP-115/OPP-115/annotations/'

# A dictionary to hold all dataframes, key will be the base filename
dataframes = {}

for annotation_filename in os.listdir(annotations_dir):
    if annotation_filename.endswith('.csv'):
        base_name = '_'.join(annotation_filename.split('_')[1:]).rsplit('.', 1)[0]

        annotation_path = os.path.join(annotations_dir, annotation_filename)

        annotation_df = pd.read_csv(annotation_path, header=None)

        dataframes[base_name] = {'categories': annotation_df[5], 'preprocessed_text': preprocessed_policies_dict.get(base_name)}



yahoo_df = dataframes['yahoo.com']
yahoo_df['categories']


0              Other
1              Other
2              Other
3              Other
4              Other
           ...      
162            Other
163    Policy Change
164            Other
165            Other
166            Other
Name: 5, Length: 167, dtype: object

In [ ]:
# Initialize a list to hold the final consolidated dataset
final_dataset = []

for base_name, data in dataframes.items():
    unique_categories_per_text = defaultdict(set)

    for category in data['categories']:
        unique_categories_per_text[data['preprocessed_text']].add(category)

    for text, categories_set in unique_categories_per_text.items():
        final_dataset.append({
            'source': base_name,
            'policies_text': text,
            'categories': list(categories_set)
        })

final_df = pd.DataFrame(final_dataset)

# Display the shape of the DataFrame and the first few rows to verify the structure
print(final_df.shape)
final_df.head()


(115, 3)


,source,policies_text,categories
0,sidearmsports.com,sidearm sports privacy policy |||privacy p...,"[Data Retention, User Access, Edit and Deletio..."
1,mlb.mlb.com,privacy policy please read this priva...,"[User Access, Edit and Deletion, Data Retentio..."
2,dcccd.edu,privacy and security commitment our p...,"[Data Retention, User Access, Edit and Deletio..."
3,uh.edu,university of houston privacy policy the ...,"[Third Party Sharing/Collection, Data Retentio..."
4,imdb.com,imdb privacy notice |||last update decembe...,"[User Access, Edit and Deletion, Data Retentio..."


In [ ]:
final_df.iloc[0]['policies_text']

"sidearm sports privacy policy     |||privacy policy      sidearm sports together with its parent company its subsidiary and its affiliate the sidearm entities own operate or provide access to several interactive web site and other interactive feature that provide an authorize link to sidearm 's privacy policy collectively our web site this privacy policy effective december 9 2014 describe the privacy practice for our web site and other application collectively refer to as the sidearm services and to any service that display this notice     |||our web sites may contain link to other website for your convenience and information we be not responsible for the privacy practice or the content of those website     |||information we collect about you      when you interact with sidearm services we collect 1 personal information you supply when you subscribe order complete a form or survey register for one of our web site enter a contest or provide your email address and 2 non personal informa

In [ ]:
# First Party Collection/Use	- maps to -> Lawfulness, Fairness, Transparency | Purpose Limitation | Data Minimization

# Third Party Sharing/Collection - maps to ->	Lawfulness, Fairness, Transparency | Purpose Limitation | Data Minimization

# User Choice/Control	 - maps to -> Lawfulness, Fairness, Transparency

# User Access, Edit, and Deletion  - maps to ->	Lawfulness, Fairness, Transparency | Accuracy

# Data Retention  - maps to ->	Storage Limitation

# Data Security  - maps to -> Integrity and Confidentiality

# Policy Change	  - maps to ->	 Lawfulness, Fairness, Transparency

# Do Not Track

# International and Specific Audiences	  - maps to ->	 Lawfulness, Fairness, Transparency

# Other	(No Direct Mapping)

label_mapping = {
    "First Party Collection/Use": [1, 2, 3],
    "Third Party Sharing/Collection": [1, 2, 3],
    "User Choice/Control": [1],
    "User Access, Edit, and Deletion": [1, 4],
    "Data Retention": [5],
    "Data Security": [6],
    "Policy Change": [1],
    "Do Not Track": [],
    "International and Specific Audiences": [1],
    "Other": []
}

# where:

# 1 is lawfulness, fairness and transparency

# 2 is purpose limitation

# 3 is data minimization

# 4 is accuracy

# 5 is storage limitation

# 6 is integrity and confidentiality

# 7 is accountability

def transform_document_labels(document_labels):
    binary_labels = [0] * 7  # There are 7 GDPR principles
    for label in document_labels:
        # Check if label has a corresponding mapping
        if label in label_mapping:
            mapped_indices = label_mapping[label]
            for idx in mapped_indices:
                binary_labels[idx - 1] = 1  # Set to 1 based on mapping
    return binary_labels

final_df['policies_labels'] = final_df['categories'].apply(transform_document_labels)

total_principles = 7

final_df['compliance_percentage'] = final_df['policies_labels'].apply(lambda x: (sum(x) / total_principles) * 100)

final_df.head()



,source,policies_text,categories,policies_labels,compliance_percentage
0,sidearmsports.com,sidearm sports privacy policy |||privacy p...,"[Data Retention, User Access, Edit and Deletio...","[1, 1, 1, 0, 1, 1, 0]",71.428571
1,mlb.mlb.com,privacy policy please read this priva...,"[User Access, Edit and Deletion, Data Retentio...","[1, 1, 1, 0, 1, 1, 0]",71.428571
2,dcccd.edu,privacy and security commitment our p...,"[Data Retention, User Access, Edit and Deletio...","[1, 1, 1, 0, 1, 1, 0]",71.428571
3,uh.edu,university of houston privacy policy the ...,"[Third Party Sharing/Collection, Data Retentio...","[1, 1, 1, 0, 1, 0, 0]",57.142857
4,imdb.com,imdb privacy notice |||last update decembe...,"[User Access, Edit and Deletion, Data Retentio...","[1, 1, 1, 0, 1, 1, 0]",71.428571


In [ ]:
# final_df.to_csv("./policies_gdpr_labels.csv", index=False)

In [25]:
# preprocessed_policies = final_df['policies_text'].tolist()
